In [31]:
from langchain_google_genai import GoogleGenerativeAI,GoogleGenerativeAIEmbeddings
from apis import google,lang_smith
from langchain.document_loaders import PyPDFLoader
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.storage import InMemoryByteStore
from langchain_community.vectorstores import Chroma
from langchain.retrievers.multi_vector import MultiVectorRetriever
from uuid import uuid4
from langchain_core.documents import Document

In [3]:
import os
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = lang_smith
os.environ['google_api_key'] = google
import os
os.environ['USER_AGENT'] = 'myagent'

In [4]:
llm = GoogleGenerativeAI(model = "gemini-pro")
embeddings =GoogleGenerativeAIEmbeddings(model="models/embedding-001")

####  Multi-representation Indexing

In [5]:
deeplearning_book= "mml-book.pdf"

In [6]:
def make_documents(book):
    docs = PyPDFLoader(book).load()
    chain = ({"doc":lambda x: x.page_content}|
    ChatPromptTemplate.from_template("Summarize the following document:\n\n{doc}")|
    llm|StrOutputParser())
    sumaries  = chain.batch(docs,{"max_concurrency": 5})
    return docs,sumaries

In [7]:
deeplearning_docs,sumaries= make_documents(deeplearning_book)

Retrying langchain_google_genai.llms._completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.llms._completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.llms._completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Failed to batch ingest runs: LangSmithConnectionError("Connection error caused failure to POST https://api.smith.langchain.com/runs/batch  in LangSmith API. Please confirm your internet connection.. ConnectionError(ProtocolError('Connection aborted.', TimeoutError('The write operation timed out')))")
Retrying langchain_google_genai.llms._completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised Re

In [9]:
len(deeplearning_docs)

417

In [17]:
print(deeplearning_docs[100].page_content)

3.10 Further Reading 95
kernel methods (Sch ¨olkopf and Smola, 2002). Kernel methods exploit the
fact that many linear algorithms can be expressed purely by inner prod-
uct computations. Then, the “kernel trick” allows us to compute these
inner products implicitly in a (potentially infinite-dimensional) feature
space, without even knowing this feature space explicitly. This allowed the
“non-linearization” of many algorithms used in machine learning, such as
kernel-PCA (Sch ¨olkopf et al., 1997) for dimensionality reduction. Gaus-
sian processes (Rasmussen and Williams, 2006) also fall into the category
of kernel methods and are the current state of the art in probabilistic re-
gression (fitting curves to data points). The idea of kernels is explored
further in Chapter 12.
Projections are often used in computer graphics, e.g., to generate shad-
ows. In optimization, orthogonal projections are often used to (iteratively)
minimize residual errors. This also has applications in machine lea

In [23]:
print(sumaries[100])

Kernel methods, like kernel-PCA and Gaussian processes, use inner product computations to implicitly work in high-dimensional feature spaces, enabling non-linear algorithms. Projections are used in computer graphics and optimization, including machine learning applications like linear regression and dimensionality reduction through PCA.


In [22]:
len(sumaries)

417

In [73]:
path ="E:\workstation\LLM\project_LL\DataScientistAssitance\embeddings"

In [74]:
vectore_store = Chroma("summaries",embedding_function=embeddings,persist_directory=path)

In [75]:
store  = InMemoryByteStore()
id_key = "doc_id"
retriever = MultiVectorRetriever(vectorstore=vectore_store,
                                 byte_store=store,
                                 id_key=id_key)

In [76]:
docs_id = [str(uuid4()) for _ in deeplearning_docs]


In [77]:
summary_docs= [Document(page_content=s,metadata={id_key:docs_id[i]}) for i,s in enumerate(sumaries)]

In [78]:
summary_docs[0]

Document(page_content='This textbook provides a comprehensive introduction to the mathematical concepts essential for understanding machine learning. It bridges the gap between mathematical and machine learning texts, offering a self-contained resource for students, professionals, and those new to the mathematics. The book covers linear algebra, analytic geometry, matrix decompositions, vector calculus, optimization, probability, and statistics. It uses these concepts to derive four central machine learning methods: linear regression, principal component analysis, Gaussian mixture models, and support vector machines. The text includes worked examples, exercises, and programming tutorials to enhance understanding and practical application.', metadata={'doc_id': '695c7dcc-4dab-4005-acd3-01e85cebd181'})

In [79]:
retriever.vectorstore.add_documents(summary_docs)

['9a2c68d4-6c98-4c8e-b64a-5512bbee69ef',
 'a8293b48-9282-4136-ac88-327bf7425a79',
 'e20e694b-af0d-4b0c-8575-863c460bbf49',
 '427c469b-ed0d-4e78-840a-f26a336a8744',
 'fa07f7b0-ce61-4fcb-a101-7010fb1547bf',
 '24a8a0c3-a077-4f77-bd54-b506c53e96af',
 'aad963df-5d20-42c4-ba3f-cacc22e54dfd',
 '82d1c76c-e590-4805-9828-b8348c1132a4',
 'e1297266-2da1-4ae2-a630-7dfdd9b9b010',
 'e89b40d1-80d3-4811-87ca-612d2cbee7a9',
 'b00990d9-5c24-4826-b2c0-9c06a345263c',
 'ea278636-dbc4-40d5-9525-67cb9af7502a',
 'cadda415-1dbb-45e2-85df-1a422bb2ea98',
 '4fb8b491-8967-43d8-a9a2-c6d315017e7e',
 '58ef858c-eb64-4c85-8b52-17a28d257d53',
 'f388b269-eb6b-45fb-a0a1-8d1944e7bd2c',
 '31a83c85-206c-401b-b211-889189a5ec60',
 '12469955-a126-49e7-87f5-d3673b37e1dc',
 '1dab79fe-648b-4855-88dd-ba887bc0b03c',
 'e8f87696-3746-4682-a562-94b3414cc389',
 '618b4d83-42aa-4a60-8861-9ba5b30413eb',
 'b5f93a5c-a0ab-4b26-a63b-f83ced1dc50a',
 '6fda2997-72cc-47a8-9c00-3ae02f981e17',
 'f52c1838-b696-4fdb-8b72-3f6c1b8feec7',
 '3c474e2a-9d14-

In [80]:
retriever.docstore.mset(list(zip(docs_id,deeplearning_docs)))


In [81]:
query = "tell me about  Matrices"
sub_docs = vectore_store.similarity_search(query,k=10)
sub_docs

[Document(page_content='Matrices are rectangular arrays of numbers that represent systems of linear equations and linear functions. They consist of rows and columns, and their dimensions are specified as (m x n), where m is the number of rows and n is the number of columns.\n\nMatrix addition is performed element-wise, while matrix multiplication is computed by multiplying each element of a row of the first matrix by the corresponding element of a column of the second matrix and summing the results. The size of the result matrix depends on the dimensions of the input matrices.', metadata={'doc_id': 'accd4e81-8655-44ca-8033-0ec91e98fb72'}),
 Document(page_content='This section provides advanced concepts in matrix theory. Normal matrices have the property A⊤A=AA⊤, and orthogonal matrices are a subset of normal matrices with A⊤A=AA⊤=I. Symmetric matrices satisfy S=S⊤ and have only real eigenvalues. Positive definite matrices are symmetric matrices that satisfy x⊤Px>0 for all x∈Rn\\{0}, ha

In [88]:
vectore = Chroma("summaries",persist_directory=path
,embedding_function=embeddings)

In [90]:
query = "tell me about  Matrices"
sub_docs = vectore.similarity_search(query,k=10)
sub_docs

[Document(page_content='Matrices are rectangular arrays of numbers that represent systems of linear equations and linear functions. They consist of rows and columns, and their dimensions are specified as (m x n), where m is the number of rows and n is the number of columns.\n\nMatrix addition is performed element-wise, while matrix multiplication is computed by multiplying each element of a row of the first matrix by the corresponding element of a column of the second matrix and summing the results. The size of the result matrix depends on the dimensions of the input matrices.', metadata={'doc_id': 'accd4e81-8655-44ca-8033-0ec91e98fb72'}),
 Document(page_content='This section provides advanced concepts in matrix theory. Normal matrices have the property A⊤A=AA⊤, and orthogonal matrices are a subset of normal matrices with A⊤A=AA⊤=I. Symmetric matrices satisfy S=S⊤ and have only real eigenvalues. Positive definite matrices are symmetric matrices that satisfy x⊤Px>0 for all x∈Rn\\{0}, ha

In [91]:
prompt = "Answer the question from the context \
    question: {question} \
        context: {context}"